# Tensor shapes in Pyro

这一部分主要介绍pyro是如何组织tensor的dimensions的。

## Distribution shapes

对于一个分布，其有两个shapes：即batch_shape和event_shape。

In [4]:
import torch
import pyro.distributions as pdist
import torch.distributions as tdist

In [12]:
pgaussion = pdist.Normal(0, 1)
tgaussion = tdist.Normal(0, 1)

In [13]:
# 注意，隔离pyro.distributions和torch.distributions行为是一致的
print("batch_shape:", pgaussion.batch_shape, tgaussion.batch_shape)
print("event_shape:", pgaussion.event_shape, tgaussion.event_shape)

batch_shape: torch.Size([]) torch.Size([])
event_shape: torch.Size([]) torch.Size([])


In [17]:
x = pgaussion.sample()
assert x.shape == pgaussion.batch_shape + pgaussion.event_shape

In [23]:
# 注意，torch.Size对象的`__add__`方法是concat到一起而不是加在一起，其类似list
torch.Size([1, 2]) + torch.Size([3])

torch.Size([1, 2, 3])

实际上，我们还有另外一个shape：samle_shape，其在使用`sample`方法的时候出现：

In [29]:
x2 = pgaussion.sample([3])
assert x2.shape == torch.Size([3]) + pgaussion.batch_shape + pgaussion.event_shape

所以我们得到的样本的shape和以上3个shape间的关系是：
```
      |      iid     | independent | dependent
------+--------------+-------------+------------
shape = sample_shape + batch_shape + event_shape
```

下面我们来具体介绍这3个shapes：

- `sample_shape`表示的是样本的维度，比如`sample_shape=[3, 4]`表示我们一共随机了产生了12个独立的样本，然后把每一个样本排列成$3\times4$的矩阵形式。
- `batch_shape`表示的是参数的维度（这里说是independent的维度，也是合理的），我们可以设置参数是`torch.tensor([2, 3])`，则`batch_shape=[2]`，也就是说会产生2个分布，这两个分布是相互独立的，其中一个分布的参数是2，另一个分布的参数是3。
- `event_shape`表示的是多维分布的那个"多维"，比如多维正态分布、Categorical distribution等等，这些维度上的各个值间不是独立的，因为会收到其他值的影响（当然这是多维正态分布的协方差不能是0）。

注意：

1. `log_prob`函数只会考虑到sample_shape和batch_shape，不管多少维的随机变量，其密度函数只会映射到$[0, 1]$区间中。
2. 一般来说，`torch.Size([])`和`torch.Size([1])`是不一样的，对于`Normal`，其`event_shape==torch.Size([])`，表示其作为一个单维度的随机变量没有`event_shape`，而对于`MultivariateNormal`，虽然我们也可以让其只有一维，但`event_shape==torch.Size([])`，表示其是一个多维随机变量，什么时候都有`event_shape`。

下面是一些例子：

In [32]:
# torch.Size()对象`__equal__`方法可以对tuple进行操作
d = pdist.Bernoulli(0.5)
assert d.batch_shape == ()
assert d.event_shape == ()
x = d.sample()
assert x.shape == ()
assert d.log_prob(x).shape == ()

In [33]:
d = pdist.Bernoulli(0.5 * torch.ones(3, 4))
assert d.batch_shape == (3, 4)
assert d.event_shape == ()
x = d.sample()
assert x.shape == (3, 4)
assert d.log_prob(x).shape == (3, 4)

In [34]:
# 还可以通过`expand()`方法来形成batch_shape，此方法的效果是在当前batch_shape的基础上进一步扩增batch_shape
d = pdist.Bernoulli(torch.tensor([0.1, 0.2, 0.3, 0.4])).expand([3, 4])
assert d.batch_shape == (3, 4)
assert d.event_shape == ()
x = d.sample()
assert x.shape == (3, 4)
assert d.log_prob(x).shape == (3, 4)

In [35]:
# Multivariate distribution有nonempty的`.event_shape`
d = pdist.MultivariateNormal(torch.zeros(3), torch.eye(3, 3))
assert d.batch_shape == ()
assert d.event_shape == (3,)
x = d.sample()
assert x.shape == (3,)
assert d.log_prob(x).shape == ()

In [37]:
# if there is nonempty sample_size，then...
x2 = d.sample([4])
assert x2.shape == (4, 3)
assert d.log_prob(x2).shape == (4,)

## Reshaping distributions

在pyro中我们可以有方法`.to_event(n)`来把某个`batch_shape`转变成`event_shape`，也就是把一些独立的分布看做是一个联合分布。这在torch中没有。

In [38]:
d = pdist.Bernoulli(0.5 * torch.ones(3, 4)).to_event(1)
assert d.batch_shape == (3,)
assert d.event_shape == (4,)
x = d.sample()
assert x.shape == (3, 4)
assert d.log_prob(x).shape == (3,)

In [52]:
# 这个转换必须从右边开始，里面的参数不是第几个，而是一共要转换几个维度
d = pdist.Bernoulli(0.5 * torch.ones(3, 4)).to_event(2)
assert d.batch_shape == ()
assert d.event_shape == (3, 4)
x = d.sample()
assert x.shape == (3, 4)
assert d.log_prob(x).shape == ()

将多个独立的分布看成一个整个的联合分布，这个操作在pyro中会非常频繁的使用。其有两个优势：

1. 这使得我们可以简单的构建一个多维分布。
2. 这可以使我们避免去使用`plate`来声明一个Multivariate distribution的独立性。

（当然，这并不一定是个好处，因为被`plate`声明了独立性后，在进行梯度估计的时候可以受益于这个独立性，而使用`to_event()`方式构建的多维独立分布在pyro的眼中并不是独立的，pyro将其作为dependent variables来看待。）

In [47]:
# 第一个版本
import pyro

x = pyro.sample("x", pdist.Normal(0, 1).expand([10]).to_event(1))
assert x.shape == (10,)

In [53]:
# 第二个版本
with pyro.plate("x_plate", 10):
    x = pyro.sample("x", pdist.Normal(0, 1))
    assert x.shape == (10,)

注意，将independent variables看做是dependent，永远都是safe的（特别是对于那些可以reparametric的distributions，此trick使得即使增加了dependence也不会降低效率）。而将dependent variables看做是independent则会导致一些问题。

## 使用`plate`来declaring independence

我们可以使用`plate`的上下文管理器来指定特定的batch dimensions是independence。这些independence可能会被inference algorithm利用从而降低Enforce estimation的variance。

注意，虽然plate可以操作某个batch dimension是independent，但其并没有将该dimension放入到event_shape中。真正将dimension放入event_shape中的操作是`to_event`，但其并没有指定independent（尽管其实就是independent）。那我们之所以需要使用`plate`来进行操作，只是为了能够利用到其inference时的trick而已。

有几种利用`plate`的方式：

第一，最简单的，直接在`sample`上使用，则其最右边的batch dimension被设定为independent

In [65]:
with pyro.plate("my_plate"):
    ss = pyro.sample("x", pdist.Normal(torch.arange(12).reshape(3, 4).float(), 1.))
    print(ss)

tensor([[-0.3545,  2.2652,  1.9722,  2.2458],
        [ 3.5141,  4.9207,  6.3201,  8.1640],
        [ 9.1618,  9.5082,  9.0513, 10.9938]])


第二，我们提供最右边的batch dimension在`plate`中作为一个参数，其效果和上面的一致

In [67]:
with pyro.plate("my_late", 4):
    ss = pyro.sample("x", pdist.Normal(torch.arange(12).reshape(3, 4).float(), 1.))
    print(ss)

tensor([[ 1.1461,  0.6381,  2.7226,  3.7322],
        [ 5.3836,  5.0731,  6.4878,  6.5001],
        [ 7.5291,  9.6671,  9.0649, 10.6666]])


In [68]:
with pyro.plate("my_late", 2):
    ss = pyro.sample("x", pdist.Normal(torch.tensor([1.0, 2.0]), 1.))
    print(ss)

tensor([1.8814, 1.2337])


In [69]:
# 这种方法的优势1：如果只有一个batch dimension，而且就想让这个batch dimension是independent，而且这个dimension就是多个distribution的
#   简单重复，则可以不需要在下面的`sample`语句中指定这个dimension
with pyro.plate("my_late", 2):
    ss = pyro.sample("x", pdist.Normal(1.0, 1.0))
    print(ss)

tensor([1.2504, 1.0534])


In [73]:
# 这种方法的优势2：可以通过嵌套的`plate`上下文管理器，来实现多个维度的independent
with pyro.plate("x_axis", 3):  # 注意，这是第-1个dimension
    with pyro.plate("y_axis", 2):  # 注意，这是第-2个dimension
        ss = pyro.sample("x", pdist.Normal(torch.arange(6).reshape(1, 2, 3).float(), 1.))
        print(ss)

tensor([[[0.5843, 2.2563, 0.0241],
         [2.7450, 3.8366, 5.4417]]])


第三，使用`plate`中的参数`dim`来指定哪一个维度是independent，这使得我们可以通过合理的组织来完成复杂的依赖关系。

In [74]:
def model1():
    a = pyro.sample("a", pdist.Normal(0, 1))
    b = pyro.sample("b", pdist.Normal(torch.zeros(2), 1).to_event(1))
    with pyro.plate("c_plate", 2):
        c = pyro.sample("c", pdist.Normal(torch.zeros(2), 1))
    with pyro.plate("d_plate", 3):
        d = pyro.sample("d", pdist.Normal(torch.zeros(3, 4, 5), 1).to_event(2))
    assert a.shape == ()           # batch_shape == (), event_shape == (), pyro independ == (), real independ == ()
    assert b.shape == (2,)         # batch_shape == (), event_shape == (2,), pyro independ == (), real independ == (2,)
    assert c.shape == (2,)         # batch_shape == (2,), event_shape == (), pyro independ == (2,), real independ == (2,)
    assert d.shape == (3, 4, 5)    # batch_shape == (3, ), event_shape == (4, 5), pyro independ == (4, 5), real independ == (3, 4, 5)

    x_axis = pyro.plate("x_axis", 3, dim=-2)
    y_axis = pyro.plate("y_axis", 2, dim=-3)
    with x_axis:
        x = pyro.sample("x", pdist.Normal(0, 1))
    with y_axis:
        y = pyro.sample("y", pdist.Normal(0, 1))
    with x_axis, y_axis:
        xy = pyro.sample("xy", pdist.Normal(0, 1))
        z = pyro.sample("z", pdist.Normal(0, 1).expand([5]).to_event(1))
    assert x.shape == (3, 1)         # batch_shape == (3, 1), event_shape == (), pyro independ == (3,), real independ == (3, 1)
    assert y.shape == (2, 1, 1)      # batch_shape == (2, 1, 1), event_shape == (), pyro independ == (2,), real independ == (2, 1, 1)
    assert xy.shape == (2, 3, 1)     # batch_shape == (2, 3, 1), event_shape == (), pyro independ == (2, 3), real independ == (2, 3, 1)
    assert z.shape == (2, 3, 1, 5)   # batch_shape == (2, 3, 1), event_shape == (5,), pyro independ == (2, 3), real independ == (2, 3, 1, 5)

In [75]:
def test_model(model, guide, loss):
    pyro.clear_param_store()
    loss.loss(model, guide)

In [77]:
from pyro.infer import Trace_ELBO

test_model(model1, model1, Trace_ELBO())

注意到，在使用`log_prob`的时候，event_shape会被sum out到一起，剩下的batch_shape（和sample_shape）会保留下来。

In [78]:
import pyro.poutine as poutine

trace = poutine.trace(model1).get_trace()
trace.compute_log_prob()
print(trace.format_shapes())

Trace Shapes:            
 Param Sites:            
Sample Sites:            
       a dist       |    
        value       |    
     log_prob       |    
       b dist       | 2  
        value       | 2  
     log_prob       |    
 c_plate dist       |    
        value     2 |    
     log_prob       |    
       c dist     2 |    
        value     2 |    
     log_prob     2 |    
 d_plate dist       |    
        value     3 |    
     log_prob       |    
       d dist     3 | 4 5
        value     3 | 4 5
     log_prob     3 |    
  x_axis dist       |    
        value     3 |    
     log_prob       |    
  y_axis dist       |    
        value     2 |    
     log_prob       |    
       x dist   3 1 |    
        value   3 1 |    
     log_prob   3 1 |    
       y dist 2 1 1 |    
        value 2 1 1 |    
     log_prob 2 1 1 |    
      xy dist 2 3 1 |    
        value 2 3 1 |    
     log_prob 2 3 1 |    
       z dist 2 3 1 | 5  
        value 2 3 1 | 5  
     log_pro

In [79]:
d = pdist.Normal(torch.ones(3, 4), 1.0).to_event(1)
x = d.sample()
d.log_prob(x)

tensor([-5.7662, -6.1560, -6.4239])

## Subsampling tensors inside a `plate`

之前说到过，`plate`的一个作用是用来做subsampling的。实际上做subsampling是需要一些条件的，比如必须是独立的，不然expected loss of subsamples并不会成比例的减小。所以一般来说subsampling只会用在样本那一层级。

但现在因为数据结构的原因，我们使用`plate`将一些维度也规定成independent，则subsampling也是可以使用的。（尽管理论上是这样，但实际上这个subsampling也只会应用到data那里）

使用时，我们需要即指定数据的总数量，而且还要指定`subsample_size`：

In [81]:
data = torch.arange(100.)
def model2():
    mean = pyro.param("mean", torch.zeros(len(data)))
    with pyro.plate("data", len(data), subsample_size=10) as ind:  # subsampling的逻辑需要在整个模型中体现出来
        assert len(ind) == 10
        batch = data[ind]
        mean_batch = mean[ind]
        x = pyro.sample("x", pdist.Normal(mean_batch, 1), obs=batch)
        assert len(x) == 10

test_model(model2, guide=lambda: None, loss=Trace_ELBO())

## Broadcasting to allow parallel enumeration

这里，pyro可以对一些discrete latent variables进行并行运算，这可以显著的降低gradient estimators’ variance。

而为了能够进行这个parallel enumeration，pyro需要知道哪些tensor dimensions可以被enumerate，所以我们需要指定一个`max_plate_nesting`。

In [85]:
from pyro.infer import config_enumerate, TraceEnum_ELBO

@config_enumerate
def model3():
    p = pyro.param("p", torch.arange(6.) / 6)
    locs = pyro.param("locs", torch.tensor([-1., 1.]))

    a = pyro.sample("a", pdist.Categorical(torch.ones(6) / 6))
    b = pyro.sample("b", pdist.Bernoulli(p[a]))
    with pyro.plate("c_plate", 4):
        c = pyro.sample("c", pdist.Bernoulli(0.3))
        with pyro.plate("d_plate", 5):
            d = pyro.sample("d", pdist.Bernoulli(0.4))
            e_loc = locs[d.long()].unsqueeze(-1)
            e_scale = torch.arange(1., 8.)
            e = pyro.sample("e", pdist.Normal(e_loc, e_scale).to_event(1))

    #                   enumerated/batch/event dims
    assert a.shape == (         6, 1, 1   )
    assert b.shape == (      2, 1, 1, 1   )
    assert c.shape == (   2, 1, 1, 1, 1   )
    assert d.shape == (2, 1, 1, 1, 1, 1,  )
    assert e.shape == (2, 1, 1, 1, 5, 4, 7)

    assert e_loc.shape   == (2, 1, 1, 1, 1, 1, 1,)
    assert e_scale.shape == (                  7,)

test_model(model3, model3, TraceEnum_ELBO(max_plate_nesting=2))

In [ ]:
p 